In [1]:
import pandas as pd
import joblib
import os
import sys

notebook_dir = os.getcwd() 
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from Database.database_manager import DatabaseManager
from Models.Functional_Classes.Glicko2.GlickoCalculator import GlickoCalculator
from Models.Functional_Classes.Glicko2.GlickoFighter import GlickoFighter
from utils.dominance_prediction import dominance_prediction
from utils.ranking_model import ranking_model
from Elo_System.Cross_Functions.create_features import prep_features

In [2]:

glicko_calculator = GlickoCalculator()

In [3]:
with DatabaseManager('../Database/fighters.db') as db:
    all_fights = db.get_fights()
    all_fighters = db.get_fighters()

all_fights = pd.DataFrame(all_fights)

In [4]:
glicko_fighters= {
    fighter.id : GlickoFighter(fighter.elo_rating, fighter.rating_deviation, fighter.rating_volatility)
    for fighter in all_fighters
}
print(glicko_fighters)

{1: GlickoFighter rating: 1401.7082521853265, rating_deviation: 114.77736456851785 volatility: 0.05997801876122141, 2: GlickoFighter rating: 1430.7972426711538, rating_deviation: 98.80822994518472 volatility: 0.059993456699954144, 3: GlickoFighter rating: 1509.390605096608, rating_deviation: 91.61082833704845 volatility: 0.05996095439545471, 4: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 5: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 6: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 7: GlickoFighter rating: 1507.2824483237612, rating_deviation: 71.58450057529885 volatility: 0.05992347934605798, 8: GlickoFighter rating: 1153.1445974898243, rating_deviation: 171.79698369671695 volatility: 0.059993961974269816, 9: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 10: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 11: GlickoFighter rating: 1500.0, rating_deviation

In [5]:
features = prep_features(all_fights)
dominance_scores = dominance_prediction(all_fights)

show_scores = pd.concat([features, dominance_scores], axis=1)
all_fights['dominance_score'] = dominance_scores

show_scores.head()

,sig_strike_differential,takedown_differential,knockdown_differential,sub_differential,total_sig_strikes,total_takedowns,total_knockdowns,total_submissions,final_round,final_time_seconds,win_method_clean_CNC,win_method_clean_DQ,win_method_clean_KO/TKO,win_method_clean_M-DEC,win_method_clean_Other,win_method_clean_Overturned,win_method_clean_S-DEC,win_method_clean_SUB,win_method_clean_U-DEC,0
0,-11,0,0,0,27,0,0,0,2,18,1,0,0,0,0,0,0,0,0,0.970353
1,11,0,1,0,21,0,1,0,1,71,0,0,1,0,0,0,0,0,0,2.276725
2,0,0,1,0,2,0,1,0,1,22,0,0,1,0,0,0,0,0,0,2.276725
3,19,0,0,0,77,0,2,0,2,183,0,0,1,0,0,0,0,0,0,1.639512
4,-6,2,0,1,116,2,0,1,3,300,0,0,0,0,0,0,1,0,0,0.873416


In [6]:
PIT_snapshots = []

for index, fight_row in all_fights.iterrows():
    red_id = fight_row['red_fighter_id']
    blue_id = fight_row['blue_fighter_id']
    winner_id = fight_row['winner_id']

    if pd.isna(red_id) or pd.isna(blue_id) or red_id not in glicko_fighters or blue_id not in glicko_fighters:
        continue

    red_fighter = glicko_fighters[red_id]
    blue_fighter = glicko_fighters[blue_id]

    dominance_score = fight_row['dominance_score']

    if winner_id == red_id:
        outcome_red = min(1.0, dominance_score / 2.0)
        outcome_blue = 1- outcome_red
    elif winner_id == blue_id:
        outcome_blue = min(1.0, dominance_score / 2.0)
        outcome_red = 1- outcome_blue
    else:
        outcome_red = 0.5
        outcome_blue = 0.5

    red_snapshot = GlickoFighter(red_fighter.rating, red_fighter.volatility,red_fighter.rating_deviation )
    blue_snapshot = GlickoFighter(blue_fighter.rating,blue_fighter.volatility, blue_fighter.rating_deviation )

    glicko_calculator.rate_1vs1(red_fighter, blue_snapshot, outcome_red)
    glicko_calculator.rate_1vs1(blue_fighter, red_snapshot, outcome_blue)

    snapshot = (
        red_snapshot.rating,
        red_fighter.rating,
        blue_snapshot.rating,
        blue_fighter.rating,
        fight_row['fight_id'],
    )

    PIT_snapshots.append(snapshot)


In [7]:
with DatabaseManager('../Database/fighters.db') as db:
    db.save_fight_elos(PIT_snapshots)

In [8]:

final_rankings, quality_scores = ranking_model(glicko_fighters, all_fighters, all_fights)
final_rankings.head(25)


[1553.1436985961382, 1359.7196252918384, 1510.8012805270491]
[1611.2802650966742, 1571.1967440365206, 1517.4988381003345, 1444.773046661149, 1717.1672473116305]
[1585.8106961151427, 1408.1859495524686, 1439.0226174653808, 1504.4701081915205, 1529.778912270311]
[1505.6748765605205, 1548.2790530267994, 1531.9599113768081, 1515.824935171548, 1414.7455190053513]
[1557.0464292008685, 1543.5276519009512]
[1435.2561008862795, 1426.4703288588582, 1370.5078454371903, 1498.9270078927832]
[1588.9012660678964]
[1455.3085493060205]
[1730.4652900917367]
[1408.165332670011, 1621.3022682108042]
[1618.4810941981796, 1532.6319737413626, 1544.5240563229634, 1483.2631417672983, 1455.259837323827]
[1570.2913249169715, 1547.1697396308866]
[1541.8037169321785]
[1542.730091552808, 1610.1640011587308, 1620.7533361038838, 1513.1006567989073, 1573.21950898653]
[1556.8134727451306]
[1630.6142349357492, 1715.6414724115727]
[1451.9791883248122, 1659.1342020447942, 1553.9430164774376, 1539.4173920305188, 1551.946762

,#,id,Name,Rating,RD,Quality Score
2169,1,3674,Khamzat Chimaev,1937.432247,82.911424,1840.522215
1017,2,1661,Francis Ngannou,1939.209284,71.654672,1790.885238
1604,3,2727,Ilia Topuria,1902.590335,80.182831,1784.396772
152,4,262,Carlos Prates,1909.174379,90.945805,1782.808898
581,5,965,Tom Aspinall,1920.997527,81.159710,1773.375566
2230,6,3780,Shavkat Rakhmonov,1871.905493,80.460780,1754.435497
2113,7,3582,Jon Jones,1820.499981,63.269455,1750.463607
1752,8,2981,Curtis Blaydes,1747.411227,63.710758,1737.226774
1477,9,2480,Jailton Almeida,1889.297022,81.562158,1735.823262
56,10,95,Kamaru Usman,1794.953843,63.525322,1731.797869


In [9]:

with DatabaseManager('../Database/fighters.db') as db:
    db.reset_fighter_ratings()
    db.update_all_fighters(glicko_fighters, quality_scores)

In [10]:
print(glicko_fighters)

{1: GlickoFighter rating: 1397.8681899969365, rating_deviation: 101.96574811837094 volatility: 0.05996970875706912, 2: GlickoFighter rating: 1431.764815558231, rating_deviation: 88.16577985347692 volatility: 0.059990513451990184, 3: GlickoFighter rating: 1506.2144327186973, rating_deviation: 81.73603226428168 volatility: 0.05994621022005067, 4: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 5: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 6: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 7: GlickoFighter rating: 1505.7782156750297, rating_deviation: 66.5292124074174 volatility: 0.059895466820579654, 8: GlickoFighter rating: 1132.470398859135, rating_deviation: 160.0609672719911 volatility: 0.05999203643615905, 9: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 10: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 11: GlickoFighter rating: 1500.0, rating_deviation: 3

In [11]:
#with DatabaseManager('../Database/fighters.db') as db:
#    db.reset_fighter_ratings()
